In [ ]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime
import smtplib
import time
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [ ]:
mymodel=load_model('mymodel.h5')
check=10000
list1=[8000,6000,4000,2000]
a=0
cap=cv2.VideoCapture(0) #0 = camera connected to system
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]      #ROI crop img
        cv2.imwrite('temp.jpg',face_img)  #saving img
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict_classes(test_image)[0][0]
        if pred==1: #not wearing mask
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
            if check==list1[a]:
                fromaddr= "covidfacemaskdetection@gmail.com"  #from
                toaddr="snehachawan1999@gmail.com"    #to
                msg=MIMEMultipart()   #divides into parts
                msg['From']=fromaddr
                msg['To']=toaddr
                msg['Subject']="NO MASK detected"
                body="Kindly wear your mask."
                msg.attach(MIMEText(body,'plain'))   #attaches to the body of mail as plain text
                server=smtplib.SMTP('smtp.gmail.com',port=587)  #for gmail we use 587 port tls
                server.starttls()
                server.login(fromaddr,"wearamask")  #password
                text=msg.as_string()
                server.sendmail(fromaddr,toaddr,text)
                server.quit()
                a=a+1

        else: #wearing mask ==0 then ...
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)

    cv2.imshow('img',img)
    check=check-1


    if cv2.waitKey(1)==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()